In [13]:
import torch
import torch_geometric
from torch_geometric.data import HeteroData
from torch_geometric import EdgeIndex
from torch import Tensor, cond
from torch.nn import Embedding

from torch_geometric.nn import GATv2Conv, Linear, to_hetero

ImportError: cannot import name 'cond' from 'torch' (/lilac/data/peer/moormana/.micromamba/envs/segger-rapids-11.8/lib/python3.11/site-packages/torch/__init__.py)

In [3]:
data = HeteroData()
data['a'].x = torch.randint(10, (5, 1))
data['b'].x = torch.randn(5, 10)
for dest in ['a', 'b']:
    edges = torch.combinations(torch.arange(5), with_replacement=True)
    idx = torch.randperm(edges.shape[0])[:8]
    data['a', 'to', dest].edge_index = EdgeIndex(edges[idx].T.contiguous())

In [9]:
class TestGAT_v1(torch.nn.Module):
    def __init__(
        self,
        num_tokens: int = 10,
        num_layers: int = 4,
        num_heads: int = 2,
        in_channels: int = 3,
        hidden_channels: int = 6,
        out_channels: int = 2,
    ):
        #TODO: add documentation
        super().__init__()

        # Initial embedding per node type
        self.lin_dict = torch.nn.ModuleDict()
        self.lin_dict['a'] = Embedding(num_tokens, in_channels)
        self.lin_dict['b'] = Linear(-1, in_channels)

        # GAT layers
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = GATv2Conv(
                in_channels=(-1, -1),
                out_channels=hidden_channels,
                heads=num_heads,
                add_self_loops=False
            )
            self.convs.append(conv)

        # Project to out dimension
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict: Tensor, edge_index_dict: Tensor) -> Tensor:
        """
        Forward pass for the Segger model.

        Args:
            x (Tensor): Node features.
            edge_index (Tensor): Edge indices.

        Returns:
            Tensor: Output node embeddings.
        """
        return x_dict

In [11]:
class TestGAT_v2(torch.nn.Module):
    def __init__(
        self,
        num_tokens: int = 10,
        num_layers: int = 4,
        num_heads: int = 2,
        in_channels: int = 3,
        hidden_channels: int = 6,
        out_channels: int = 2,
    ):
        #TODO: add documentation
        super().__init__()

        # Embedding for 'tx' (transcript) nodes
        self.emb = Embedding(num_tokens, in_channels)

        # Linear layer for boundary (non-token) nodes
        self.lin = Linear(-1, in_channels, bias=False)

        # GAT layers
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = GATv2Conv(
                in_channels=(-1, -1),
                out_channels=hidden_channels,
                heads=num_heads,
                add_self_loops=False
            )
            self.convs.append(conv)

        # Project to out dimension
        self.out = Linear(hidden_channels, out_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        """
        Forward pass for the Segger model.

        Args:
            x (Tensor): Node features.
            edge_index (Tensor): Edge indices.

        Returns:
            Tensor: Output node embeddings.
        """
        
        
        def emb_fn(x: torch.Tensor):
            return self.emb(x.int())
        def lin_fn(x: torch.Tensor):
            return self.lin(x.float())

        is_1d = (x.ndim == 1).float()
        x = x[:, None]
        x = torch.cond(is_1d, emb_fn, lin_fn, x)
        x = x.squeeze()
        
        # All but final layer
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = x.relu()

        # Last layer
        x = self.convs[-1](x, edge_index)
        x = self.out(x)

        return x

In [12]:
to_hetero(TestGAT_v2(), data.metadata()).forward

AttributeError: module 'torch' has no attribute 'cond'